## Custom model 
In this notebook we describe how you can implement your own model. The model which can not be built by "block models builders" from framework.

### Notebook consists of next main stages:
1. Setup the environment
1. Define, create the model and move it to search space
1. Prepare dataset and create dataloaders
1. Check pretrain, search and tune phases

## 0. Setup the environment
First, let's set up the environment and common imports.

In [ ]:
# Copy your license file to $HOME/.enot/enot.lic or set full path to licence file 
# through environment variable ENOT_LIC_FILE
#
# Important note: no quotes
# %env ENOT_LIC_FILE=/FULL/PATH/TO/your_company.lic

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
# You should change to free GPU
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.models import SearchSpaceModel
from enot.models.mobilenet import MobileNetBaseHead
from enot.models.mobilenet import MobileNetBaseStem
from enot.models.operations import SearchableMobileInvertedBottleneck
from enot.models.operations import SearchableFuseableSkipConv
from enot.models.operations import SearchVariantsContainer
from enot.phases import pretrain
from enot.phases import search
from enot.phases import train

from enot_utils.metric_utils import accuracy
from enot_utils.schedulers import WarmupScheduler

from tutorial_utils.checkpoints import download_getting_started_pretrain_checkpoint
from tutorial_utils.dataset import create_imagenette_dataloaders

### In the next cell we setup all required dirs

* `ENOT_HOME_DIR` - is root dir for all other dirs
* `ENOT_DATASETS_DIR` - is root dir for datasets (imagenette2)
* `PROJECT_DIR` - is root dir for output data (checkpoints, logs...) of current tutorial

In [ ]:
ENOT_HOME_DIR = Path.home() / '.enot'
ENOT_DATASETS_DIR = ENOT_HOME_DIR / 'datasets'
PROJECT_DIR = ENOT_HOME_DIR / 'custom_model'

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

## 1. Prepare dataset and create dataloaders

In [ ]:
dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=ENOT_DATASETS_DIR, 
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
)

## 2. Create the model and move it to search space
To create your custom model you should build it as common pytorch model with the following rules:
1. Search variants must be placed in `SearchVariantsContainer` module.
2. All `SearchVariantsContainer` must contain the same number of operations. Operations of different containers can be different, but all containers must contain the same number of operations. **This restriction will be removed in the future versions.**
3. Every operation in SearchVariantsContainer should be created as in "Tutorial - adding custom ops" or you can use predefined operations from framework (`enot.model.operations`).

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.stem = MobileNetBaseStem(
            in_channels=3
        )
        self.body = nn.ModuleList([
            # 3 blocks with 3 search variants in every block 
            self.build_search_variants_1(16, 24, 2),
            self.build_search_variants_1(24, 24, 1),
            self.build_search_variants_1(24, 24, 1),
            # 2 fixed blocks
            self.build_mib_k3_e6(24, 32, 2),
            self.build_mib_k3_e6(32, 32, 1),
            # 3 blocks with 3 search variants in every block
            self.build_search_variants_1(32, 64, 2),
            self.build_search_variants_1(64, 64, 1),
            self.build_search_variants_1(64, 64, 1),
            # 1 fixed block
            self.build_mib_k3_e6(64, 96, 1),
            # 2 blocks with 3 search variants in every block
            self.build_search_variants_2(96, 160, 2),
            self.build_search_variants_2(160, 160, 1),
            # 1 block with 3 search variants
            self.build_search_variants_2(160, 320, 1),
        ])
        self.head = MobileNetBaseHead(
            bottleneck_channels=320,
            last_channels=1280, 
            num_classes=10,
        )
    
    @staticmethod
    def build_search_variants_1(in_channels, out_channels, stride):
        return SearchVariantsContainer([
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=5,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=7,
                stride=stride,
                expand_ratio=3,
            ),
        ])

    @staticmethod
    def build_search_variants_2(in_channels, out_channels, stride):
        return SearchVariantsContainer([
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=5,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableFuseableSkipConv(
                in_channels=in_channels,
                out_channels=out_channels,
                stride=stride,
            ),
        ])

    @staticmethod
    def build_mib_k3_e6(in_channels, out_channels, stride):
        return SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            )

    def forward(self, x):
        x = self.stem(x)
        
        for block in self.body:
            x = block(x)
            
        x = self.head(x)
        
        return x
    
model = MyModel()
# move model to search space
search_space = SearchSpaceModel(model, train_loader=dataloaders['pretrain_train_dataloader']).cuda()

## 4. Check pretrain, search and tune phases

**IMPORTANT:**<br>
`N_EPOCHS` of pretrain should be in range >= 100, if you wanna get good pretrain. In this tutorial we set `N_EPOCHS` = 3 and just check phases. 

In [ ]:
# define directory for text logs and tensorboard logs
pretrain_dir = PROJECT_DIR / 'pretrain'
pretrain_dir.mkdir(exist_ok=True)

N_EPOCHS = 3
N_WARMUP_EPOCHS = 1
len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train*N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train*N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

pretrain(
    search_space=search_space,
    exp_dir=pretrain_dir,
    train_loader=dataloaders['pretrain_train_dataloader'],
    valid_loader=dataloaders['pretrain_validation_dataloader'],
    optimizer=optimizer,
    scheduler=scheduler,
    metric_function=accuracy,
    loss_function=loss_function,
    epochs=N_EPOCHS,
)

In [ ]:
# define directory for text logs and tensorboard logs
search_dir = PROJECT_DIR / 'search'
search_dir.mkdir(exist_ok=True)

optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)

search(
    search_space=search_space,
    exp_dir=search_dir,
    search_loader=dataloaders['search_train_dataloader'],
    valid_loader=dataloaders['search_validation_dataloader'],
    optimizer=optimizer,
    loss_function=loss_function,
    metric_function=accuracy,
    latency_loss_weight=2.0e-3,
    epochs=5,
)

In [ ]:
# get regular model with best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [ ]:
# define directory for text logs and tensorboard logs
tune_dir = PROJECT_DIR / 'tune'
tune_dir.mkdir(exist_ok=True)

optimizer = RAdam(best_model.parameters(), lr=5e-3, weight_decay=4e-5)

train(
    model=best_model,
    exp_dir=tune_dir,
    train_loader=dataloaders['tune_train_dataloader'],
    valid_loader=dataloaders['tune_validation_dataloader'],
    optimizer=optimizer,
    loss_function=loss_function,
    metric_function=accuracy,
    epochs=5,
)